In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset")

print("Path to dataset files:", path)

100%|██████████| 77.2M/77.2M [00:04<00:00, 17.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset/versions/1


## Importar Librerias

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('punkt_tab')
nltk.download("punkt")
nltk.download("stopwords")
import re
from nltk.corpus import stopwords
from collections import defaultdict


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Cargar datasets

In [7]:
#Ruta del Corpus_reviews
path = "/root/.cache/kagglehub/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset/versions/1/rotten_tomatoes_critic_reviews.csv"

df_reviews = pd.read_csv(path)

print("Documentos cargados:", len(df_reviews))
display(df_reviews.head())

Documentos cargados: 1130017


,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [12]:
#Ruta del Corpus_movies
path = "/root/.cache/kagglehub/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset/versions/1/rotten_tomatoes_movies.csv"

df_movies = pd.read_csv(path)

print("Documentos cargados:", len(df_movies))
display(df_movies.head())

Documentos cargados: 17712


,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


## Preprocesamiento


In [9]:
#Limpiar columnas de df_reviews
df_reviews = df_reviews.drop(['critic_name', 'top_critic', 'publisher_name', 'review_date'], axis=1)
display(df_reviews.head())

,rotten_tomatoes_link,review_type,review_score,review_content
0,m/0814255,Fresh,NaN,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Fresh,NaN,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,Fresh,NaN,With a top-notch cast and dazzling special eff...
3,m/0814255,Fresh,3.5/5,Whether audiences will get behind The Lightnin...
4,m/0814255,Rotten,NaN,What's really lacking in The Lightning Thief i...


In [13]:
#Limpiar columnas de df_movies
df_movies = df_movies.drop(['content_rating', 'genres', 'directors', 'authors','actors', 'original_release_date', 'production_company', 'streaming_release_date', 'runtime'], axis=1)
display(df_movies.head())

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [18]:
#Funcion para preprocesar el dataframe
def clean_and_tokenize(text):
    #Asegurarse de que el texto es una cadena
    if not isinstance(text, str):
        text = str(text)  # Convertir a cadena si no lo es

    #Convertir a minúsculas
    text = text.lower()

    #Eliminar caracteres no alfabéticos
    text = re.sub(r"[^a-záéíóúñü\s]", " ", text)

    #Tokenización
    tokens = nltk.word_tokenize(text, language="spanish")

    #Stopwords en español
    stop_es = set(stopwords.words("spanish"))

    #Remover tokens muy cortos y stopwords
    tokens = [t for t in tokens if len(t) > 2 and t not in stop_es]

    return tokens


In [19]:
#Aplicar preprocesamiento
df_reviews['tokens'] = df_reviews['review_content'].apply(clean_and_tokenize)
display(df_reviews.head())

,rotten_tomatoes_link,review_type,review_score,review_content,tokens
0,m/0814255,Fresh,NaN,A fantasy adventure that fuses Greek mythology...,"[fantasy, adventure, that, fuses, greek, mytho..."
1,m/0814255,Fresh,NaN,"Uma Thurman as Medusa, the gorgon with a coiff...","[uma, thurman, medusa, the, gorgon, with, coif..."
2,m/0814255,Fresh,NaN,With a top-notch cast and dazzling special eff...,"[with, top, notch, cast, and, dazzling, specia..."
3,m/0814255,Fresh,3.5/5,Whether audiences will get behind The Lightnin...,"[whether, audiences, will, get, behind, the, l..."
4,m/0814255,Rotten,NaN,What's really lacking in The Lightning Thief i...,"[what, really, lacking, the, lightning, thief,..."


### Generar indice invertido

In [20]:
#Función para construir el índice invertido
from collections import defaultdict, Counter

def construir_indice_invertido(df):
    indice_invertido = defaultdict(dict)

    for index, row in df.iterrows():
        id_documento = index
        tokens = row['tokens']   # ESTO YA ES UNA LISTA

        # Contar frecuencia de cada token
        frecuencias = Counter(tokens)

        for palabra, freq in frecuencias.items():
            indice_invertido[palabra][id_documento] = freq

    return indice_invertido


In [21]:
#Construir el índice invertido
indice = construir_indice_invertido(df_reviews)

## Construir el sistema
Para este sistema de recuperacion usaremos dos metodos para evaluar la similitud de las criticas con consultas de texto ingresadas por el usuario

### Similitud Jaccard

In [22]:
#Funcion de similitud jaccard
def similitud_jaccard(lista1, lista2):
    set1 = set(lista1)
    set2 = set(lista2)

    interseccion = len(set1.intersection(set2))
    union = len(set1.union(set2))

    if union == 0:
        return 0.0

    return interseccion / union

In [23]:
#Funcion para recuperacion de datos
def recuperar_por_jaccard(df, query_tokens, top_k=10):
    resultados = []

    for index, row in df.iterrows():
        tokens_doc = row['tokens']
        score = similitud_jaccard(query_tokens, tokens_doc)
        resultados.append((index, score))

    # Ordenar por score de mayor a menor
    resultados = sorted(resultados, key=lambda x: x[1], reverse=True)

    return resultados[:top_k]


In [24]:
#Funcionalidad
consulta = ["space", "travel", "movie"]

resultados = recuperar_por_jaccard(df_reviews, consulta, top_k=5)

#Visualizar resultados
for doc_id, score in resultados:
    print(f"Doc: {doc_id}, Similitud: {score}")
    print(df_reviews.loc[doc_id, 'review_content'])
    print("-----")

Doc: 954778, Similitud: 0.3333333333333333
My #1 movie of 2017
-----
Doc: 509988, Similitud: 0.2857142857142857
An awfully sweet little time-travel movie.
-----
Doc: 847006, Similitud: 0.2857142857142857
Space Cowboys is the summer's best movie.
-----
Doc: 847090, Similitud: 0.2857142857142857
Space Cowboys is the summer's best movie.
-----
Doc: 848368, Similitud: 0.2857142857142857
Spark: A Space Tail is a real mishmash of a movie.
-----


### Similitud de Coseno utilizando TF-ID

In [25]:
#Matriz TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir lista de tokens a texto
df_reviews['texto'] = df_reviews['tokens'].apply(lambda x: " ".join(x))

# Crear el vectorizador TF-IDF
vectorizador = TfidfVectorizer()

# Ajustar y transformar documentos
tfidf_matrix = vectorizador.fit_transform(df_reviews['texto'])

In [26]:
#Calcular Similitud Coseno
from sklearn.metrics.pairwise import cosine_similarity

def recuperar_por_coseno(query_tokens, tfidf_matrix, vectorizador, top_k=10):
    # Convertir lista de tokens a texto igual que los docs
    query_text = " ".join(query_tokens)

    # Vectorizar la consulta con el mismo vocabulario
    query_vec = vectorizador.transform([query_text])

    # Calcular similitud del coseno entre la consulta y todos los documentos
    similitudes = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Obtener los documentos con mayor similitud
    top_docs = similitudes.argsort()[::-1][:top_k]

    # Retornar (doc_id, score)
    resultados = [(doc_id, similitudes[doc_id]) for doc_id in top_docs]

    return resultados

In [27]:
#Funcionalidad
consulta = ["space", "travel", "movie"]

resultados = recuperar_por_coseno(consulta, tfidf_matrix, vectorizador, top_k=5)

#Visualizacion
for doc, score in resultados:
    print(f"Doc {doc} → Score: {score:.4f}")
    print(df_reviews.loc[doc, 'review_content'])
    print("-----")

Doc 346898 → Score: 0.6668
The special effects depicting space travel in this movie are loud, violent and claustrophobic, making the dangers of space travel seem more personal.
-----
Doc 110181 → Score: 0.5599
Ad Astra isn't so much about the science of space and space travel as it is about the spaces between people.
-----
Doc 464911 → Score: 0.5116
Combination science fiction film and thriller is an early look at space travel on film.
-----
Doc 141461 → Score: 0.4944
The film helps audiences travel into space with such ease. [Full Review in Spanish]
-----
Doc 110398 → Score: 0.4704
Fantastic realistic space travel with a 'Mad Max' tone, and cinematography that is (out-of-this-world), pun intended- I enjoyed the sci-fi trip to space.
-----


### Consultas de texto libre

In [28]:
#Funcion para procesar una Query entrante
def preprocesar_consulta(query):
    query = query.lower()
    query = re.sub(r'[^a-zñáéíóúü\s]', ' ', query)
    query = re.sub(r'\s+', ' ', query).strip()

    tokens = query.split()
    return tokens

In [29]:
#Consultar con TF-IDF + Coseno
def buscar_coseno_texto_libre(query, vectorizador, tfidf_matrix, df, top_k=10):
    tokens = preprocesar_consulta(query)
    query_text = " ".join(tokens)

    query_vec = vectorizador.transform([query_text])

    from sklearn.metrics.pairwise import cosine_similarity
    similitudes = cosine_similarity(query_vec, tfidf_matrix).flatten()

    top_docs = similitudes.argsort()[::-1][:top_k]
    resultados = [(doc, similitudes[doc]) for doc in top_docs]

    return resultados

In [30]:
def buscar_jaccard_texto_libre(query, df, top_k=10):
    tokens = preprocesar_consulta(query)

    resultados = []

    for index, row in df.iterrows():
        score = similitud_jaccard(tokens, row['tokens'])
        resultados.append((index, score))

    resultados = sorted(resultados, key=lambda x: x[1], reverse=True)
    return resultados[:top_k]

## Simulacion de consultas con una query de texto libre

In [41]:
#Ingreso de la Query
query_usuario = input("Ingrese su consulta de texto libre: ")

Ingrese su consulta de texto libre: movie about jungle temple


In [42]:
#Para vizualizar el nombre de la pelicula consultada realizamos un merge entre ambos datarames usando rotten_tomatoes_link como clave
df_merge = df_reviews.merge(df_movies[['rotten_tomatoes_link', 'movie_title']],
                                          on='rotten_tomatoes_link',
                                          how='left')

In [43]:
#Resultados query_usuario TF-IDF Coseno
resultados = buscar_coseno_texto_libre(
    query_usuario,
    vectorizador,
    tfidf_matrix,
    df_reviews
)

#Visualizacion
for doc, score in resultados:
    #Obtener el nombre de la película
    movie_title = df_merge.loc[doc, 'movie_title']

    # Si la película no se encuentra, asignar un mensaje adecuado
    if pd.isna(movie_title):
        movie_title = "Título no disponible"

    print(f"Movie Title: {movie_title} → Score: {score:.4f}")

    # Imprimir el contenido de la reseña
    print(df_reviews.loc[doc, 'review_content'])
    print("-----")

Movie Title: Captain January → Score: 0.5631
So-so Temple vehicle.
-----
Movie Title: The Little Princess → Score: 0.5319
Good Temple vehicle.
-----
Movie Title: Jungle 2 Jungle → Score: 0.4700
Jungle2Jungle is adequate family fare, not much more.
-----
Movie Title: Heidi → Score: 0.4607
not bad, as Temple fare goes
-----
Movie Title: Tarzan, the Ape Man → Score: 0.4548
about as erotic as a Shirley Temple musical
-----
Movie Title: Baby: Secret of the Lost Legend → Score: 0.4527
Not much to say about this hackneyed jungle movie.
-----
Movie Title: The Jungle Book → Score: 0.4464
Simply put, The Jungle Book 2016 is a real swinging time in the jungle.
-----
Movie Title: Jungle → Score: 0.4238
Jungle most definitely expands on the meaning of the expression "it's a jungle out there".
-----
Movie Title: Heidi → Score: 0.4151
A better than usual Shirley Temple film.
-----
Movie Title: Heidi → Score: 0.4137
Call me a sap, but Temple gets to me.
-----


In [44]:
#Resultados query_usuario jaccard
resultados = buscar_jaccard_texto_libre(query_usuario, df_reviews, top_k=5)

#Visualizacion
for doc, score in resultados:
    #Obtener el nombre de la película
    movie_title = df_merge.loc[doc, 'movie_title']

    # Si la película no se encuentra, asignar un mensaje adecuado
    if pd.isna(movie_title):
        movie_title = "Título no disponible"

    print(f"Movie Title: {movie_title} → Score: {score:.4f}")

    # Imprimir el contenido de la reseña
    print(df_reviews.loc[doc, 'review_content'])
    print("-----")

Movie Title: 8MM → Score: 0.4000
An exploitation movie about an exploitation movie.
-----
Movie Title: The Darjeeling Limited → Score: 0.4000
It's a movie about luggage.
-----
Movie Title: After Fall, Winter → Score: 0.3333
A mixed-up movie about mixed-up people.
-----
Movie Title: After Love (L'économie du couple) → Score: 0.3333
An irritating movie about irritating people.
-----
Movie Title: Baby: Secret of the Lost Legend → Score: 0.3333
Not much to say about this hackneyed jungle movie.
-----


## Analisis de resultados por relevancia.

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
#Ingresar la Query
query = input("Ingresa tu consulta de texto: ")

# Preprocesar igual que tus documentos
query_tokens = preprocesar_consulta(query)

print("\nQuery procesada:", query_tokens)

Ingresa tu consulta de texto: romance and death

Query procesada: ['romance', 'and', 'death']


In [51]:
# JACCARD
def similitud_jaccard(query_tokens, df):
    resultados = []

    for doc_id, row in df.iterrows():
        doc_tokens = set(row["tokens"])
        query_set = set(query_tokens)

        inter = len(doc_tokens & query_set)
        union = len(doc_tokens | query_set)

        score = inter / union if union > 0 else 0
        resultados.append((doc_id, score))

    return sorted(resultados, key=lambda x: x[1], reverse=True)


ranking_jaccard = similitud_jaccard(query_tokens, df_reviews)

In [52]:
#COSENO TF-IDF
def similitud_coseno_tfidf(query):
    query_vec = vectorizador.transform([query])
    scores = cosine_similarity(query_vec, tfidf_matrix)[0]

    ranking = list(enumerate(scores))
    return sorted(ranking, key=lambda x: x[1], reverse=True)


ranking_coseno = similitud_coseno_tfidf(query)

In [53]:
#MOSTRAR RANKINGS
def mostrar_ranking(nombre, ranking, top=10):
    print(f"\n========== RANKING: {nombre} ==========")
    for doc_id, score in ranking[:top]:
        #Obtener el nombre de la película
        movie_title = df_merge.loc[doc_id, 'movie_title']

        # Si la película no se encuentra, asignar un mensaje adecuado
        if pd.isna(movie_title):
            movie_title = "Título no disponible"

        print(f"Movie Title: {movie_title} → Score: {score:.4f}")

print(f"Query_usuario:  {query}")

mostrar_ranking("Jaccard", ranking_jaccard)
mostrar_ranking("Coseno TF-IDF", ranking_coseno)


Query_usuario:  romance and death

========== RANKING: Jaccard ==========
Movie Title: Eternal Sunshine of the Spotless Mind → Score: 0.4000
Movie Title: John Wick → Score: 0.4000
Movie Title: The Disappearance of Eleanor Rigby: Them → Score: 0.4000
Movie Title: Sabrina → Score: 0.3333
Movie Title: Endless → Score: 0.3333
Movie Title: Everest → Score: 0.3333
Movie Title: Final Destination → Score: 0.3333
Movie Title: The Final Destination → Score: 0.3333
Movie Title: La Fille Coupée en Deux (The Girl Cut in Two) (A Girl Cut in Two) → Score: 0.3333
Movie Title: Kate & Leopold → Score: 0.3333

========== RANKING: Coseno TF-IDF ==========
Movie Title: Life of the Party → Score: 0.5309
Movie Title: Alive Inside → Score: 0.5109
Movie Title: Slow Burn → Score: 0.5017
Movie Title: Slow Burn → Score: 0.5017
Movie Title: A Band Called Death → Score: 0.5017
Movie Title: Black Swan → Score: 0.4922
Movie Title: The Lake House → Score: 0.4733
Movie Title: For Love of the Game → Score: 0.4602
Movie 

# Conclusión

## TF-IDF
Generalmente es más adecuado para medir la relevancia de documentos con respecto a una consulta, ya que tiene en cuenta la frecuencia de las palabras.

## Jaccard
Funciona mas cuando solo te interesa saber la superposición exacta de palabras, sin tener en cuenta su frecuencia.



# Posibles mejoras

## Combinacion de tecnicas
Combinar ambas tenicas en un solo sistema podria brindar una mayor presicion asigando un peso a cada tecnica segun el caso de la consula realizada

## Aprendizaje automatico
Podemos uilizar atributos puntuales de cada tecnica como los puntajes de TF_IDF y las caracteristicas de JACCARD para un modelo de aprendizaje automatico